Import tensorflow and numpy

In [1]:
import tensorflow as tf
import numpy as np

Reset default graph

In [2]:
tf.reset_default_graph()

Some settings

In [3]:
#
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_outputs = 10
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None), name='y')

To avoid repeating codes, we use Python's partial() function to define  my_dense_layer function. This function would return a dense layer using ReLU activation function and l1 regularization with scale of 0.01

In [4]:
from functools import partial

In [5]:
my_dense_layer = partial(tf.layers.dense, activation=tf.nn.relu, 
                         kernel_regularizer=tf.contrib.layers.l1_regularizer(0.01))

In namescope 'dnn', define hidden1 and hidden2 by my_dense_layer. Then define logits as the output layer with no activation.

In [6]:
with tf.name_scope('dnn'):
    hidden1 = my_dense_layer(X, n_hidden1, name='hidden1')
    hidden2 = my_dense_layer(hidden1, n_hidden2, name='hidden2')
    logits  = tf.layers.dense(hidden2, n_outputs, activation=None, name='logits')

In namescope 'loss', define the loss function. Note that loss should be the sum of the base loss and all regularization loss.

In [7]:
with tf.name_scope('loss'):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    base_loss = tf.reduce_mean(entropy, name='entropy')
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n([base_loss]+reg_loss, name='loss')